<center>
    <h1>Primer model per a detectar vehicles en DashCam</h1>
    <h3>S'agafa vídeo de youtube https://www.youtube.com/watch?v=igt9Br90Qj8 [1]</h3>
    <h4>es mira de practicar segons altres tutorials de youtube amb Yolo</h4>
</center>

---

Es segueixen les instruccions https://www.youtube.com/watch?v=ab5mBmXX0mQ&t=599s [1]



---
<small>

<a name="note_paper"></a>[1] Afegir a bibliografia

</small>


In [4]:
!pip install opencv-python
!pip install scikit-learn
!pip install matplotlib
!pip install torchinfo
!pip install ultralytics

In [1]:
# Settings for autoreloading.

%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import*

model = YOLO('yolov8s.pt')

In [3]:
cap2 = cv2.VideoCapture('dashcamexample.mp4')

In [4]:
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 
              'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [5]:
cap = cap2

if not cap.isOpened():
    print("Error: No es pot obrir la càmera o el vídeo.")
    exit()

tracker = Tracker()  # Si tens una classe Tracker, inicialitza-la aquí.

count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: No es pot llegir el frame.")
        break
    count += 1
    frame = cv2.resize(frame, (1440, 1080))  # Redimensionar el frame

    results = model.predict(frame)  # Predicció amb YOLO
    a = results[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")
    #print(px)

    list = []
    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]  # Assegura't que 'class_list' estigui definit
        if 'car' in c:
            list.append([x1, y1, x2, y2])

    bbox_id = tracker.update(list)  # Actualitza el tracker

    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = int(x3 + x4) // 2
        cy = int(y3 + y4) // 2

        cv2.circle(frame, (cx,cy),4,(0,0,255),-1)
        cv2.rectangle(frame,(x3,y3),(x4,y4),(0,0,255),2)

        # Aquí pots afegir més codi per dibuixar el tracking o la informació a la imatge

    cv2.imshow("frames", frame)

    if cv2.waitKey(10) & 0xFF == 27:  # Espera 1 mil·lisegon, ESC per sortir
        break

# Alliberar la càmera i tancar finestres
cap.release()
cv2.destroyAllWindows()


0: 480x640 7 persons, 6 cars, 2 motorcycles, 1 truck, 47.4ms
Speed: 4.8ms preprocess, 47.4ms inference, 97.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 persons, 8 cars, 2 motorcycles, 1 truck, 14.3ms
Speed: 3.9ms preprocess, 14.3ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 6 persons, 9 cars, 3 motorcycles, 14.4ms
Speed: 3.1ms preprocess, 14.4ms inference, 2.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 7 persons, 9 cars, 3 motorcycles, 14.5ms
Speed: 3.3ms preprocess, 14.5ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 persons, 8 cars, 3 motorcycles, 14.7ms
Speed: 3.0ms preprocess, 14.7ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 8 cars, 4 motorcycles, 14.4ms
Speed: 3.0ms preprocess, 14.4ms inference, 3.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 6 cars, 2 motorcycles, 14.5ms
Speed: 3.0ms preprocess, 14.